In [ ]:
import os
from langchain.llms import OpenAI

**Note:** Worked with Chanteria Milner on this assignment.
<img src="misc/syllabus_segment.png" style="width:400px">

# Constants, Utility Functions, and Data Importing

In [ ]:
# Constants and clients
GPT_MODEL = "gpt-3.5-turbo"
MAX_CHAR_LEN = 5000000
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
# Utility functions

In [ ]:
# Data importing

## <font color="red">*Exercise 1*</font>

<font color="red">Construct cells immediately below this that use ConvoKit to analyze a Corpus other 
than 'subreddit-Cornell', including at least one function you find in the package 
not used above. You can also generate a ConvoKit Corpus from your own dataset based 
on [their Corpus from .txt files tutorial](https://github.com/CornellNLP/Cornell-Conversational-Analysis-Toolkit/blob/master/examples/converting_movie_corpus.ipynb) or [their Corpus from pandas tutorial](https://github.com/CornellNLP/Cornell-Conversational-Analysis-Toolkit/blob/master/examples/corpus_from_pandas.ipynb), but that may 
be time-consuming for a weekly assignment.

## <font color="red">*Exercise 2*</font>

<font color="red">Construct cells immediately below this that perform a similar social 
similarity or influence analysis on a dataset relevant to your final project (__or 
one from ConvoKit__). Create relationships between actors in a network based on your 
dataset (e.g., person to person or document to document), and perform analyses that 
interrogate the structure of their interactions, similarity, and/or influence on 
one another. (For example, if relevant to your final project, you could explore 
different soap operas, counting how many times a character may have used the word 
love in conversation with another character, and identify if characters in love 
speak like each other. Or do opposites attract?) What does that analysis and its 
output reveal about the relative influence of each actor on others? What does it 
reveal about the social game being played?

<font color="red">Stretch 1:
Render the social network with weights (e.g., based on the number of scenes in 
which actors appear together), then calculate the most central actors in the 
`show.Realtime` output can be viewed in shell.

<font color="red">Stretch 2:
Implement more complex measures of similarity based on the papers you have read.

## <font color="red">*Exercise 3*</font>

<font color="red">Review the documentation for tools and agents from LangChain. Use at 
least two tools with appropriate agents discovered during your review to construct a 
chain addressing questions pertinent to your final project. If your project dataset 
is unsuitable for this task, select an alternative small-sized dataset for 
implementation.

## <font color="red">*Exercise 4*</font>

<font color="red">Use LangChain(you're welcome to not use it) to set up conversations with LLM 
agents for questions related to your final project (if relevant), or think of a 
scenario that a simulated conversation could be useful to answer a research question 
and find a dataset to implement it. What does it reveal about the social game involved 
with your dataset?

<font color="red"> Stretch: Use the idea of memory retrieval(or other methods) to design better 
templates for the LLM conversation.